In [5]:
import src.utils.regime as regime
import src.scanner as scanner
import src.floor_ceiling_regime as sfcr

In [7]:
sp500_wiki = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
ticks_, _ = scanner.get_wikipedia_stocks(sp500_wiki)
scanner_ = scanner.StockDataGetter(
    data_getter_method=lambda s: scanner.yf_get_stock_data(s, days=58, interval="15m"),
).yield_strategy_data(
    bench_symbol="SPY",
    symbols=ticks_[:2],
    strategy=lambda pdf_, bdf_: (
        sfcr.fc_scale_strategy(
            price_data=scanner.data_to_relative(pdf_, bdf_),
            distance_pct=0.05,
            retrace_pct=0.05,
            swing_window=63,
            sw_lvl=3,
            regime_threshold=0.5,
            trail_offset_pct=0.01,
            r_multiplier=1.5,
            entry_lvls=None,
            highest_peak_lvl=3,
            side_only=None,
        )
    ),
    expected_exceptions=(regime.NotEnoughDataError, sfcr.NoEntriesError)
)
stat_overview_ = scanner.run_scanner(
    scanner=scanner_,
    stat_calculator=lambda data_, entry_signals_: sfcr.calc_stats(
        data_,
        entry_signals_,
        min_periods=50,
        window=200,
        percentile=0.05,
        limit=5,
        freq='15T',
    )
)
stat_overview_

KeyboardInterrupt: 

In [6]:
import src.money_management as smm

res = smm.eqty_risk_shares()